In [61]:
import torch
from torch.utils.data import DataLoader,Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt


In [62]:
torch.manual_seed(42)

In [63]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [64]:

df=pd.read_excel("fmnist_small.xlsx")
df.head(5)

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,9,0,0,0,0,0,0,0,0,0,...,0,7,0,50,205,196,213,165,0,0
1,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,142,142,142,21,0,3,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,0,0,0,0,0,0,0,0,0,...,213,203,174,151,188,10,0,0,0,0


In [65]:
x=df.iloc[:,1:].values
y=df.iloc[:,0].values

In [66]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
x_train

array([[ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ..., 16,  0,  0]])

In [67]:
y_train

array([7, 6, 7, ..., 0, 8, 6])

In [68]:
x_test

array([[ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  1,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ..., 51,  0,  0]])

In [69]:
y_test

array([7, 0, 5, ..., 7, 1, 2])

In [70]:
x_train=x_train/255.0
x_test=x_test/255.0

In [71]:
x_train.shape

(4800, 784)

In [72]:
class custom(Dataset):
  def __init__(self,features,labels):
    self.features=torch.tensor(features,dtype=torch.float32)
    self.labels=torch.tensor(labels,dtype=torch.long)
  def __len__(self):
    return len(self.features)
  def __getitem__(self,index):
    return self.features[index],self.labels[index]

In [73]:
train_dataset=custom(x_train,y_train)


In [74]:
train_dataset[0]

(tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0

In [75]:
test_dataset=custom(x_test,y_test)

In [76]:
test_dataset

In [77]:
class myNN(nn.Module):
  def __init__(self,input_dim,output_dim,num_hidden_layers,neurons_per_layer,dropout_rate):
    super().__init__()
    layers=[]
    for i in range(num_hidden_layers):
      layers.append(nn.Linear(input_dim,neurons_per_layer))
      layers.append(nn.BatchNorm1d(neurons_per_layer))
      layers.append(nn.ReLU())
      layers.append(nn.Dropout(dropout_rate))
      input_dim=neurons_per_layer

    layers.append(nn.Linear(neurons_per_layer,output_dim))
    self.model=nn.Sequential(*layers)

  def forward(self,x):
    return self.model(x)



In [78]:
!pip install optuna

In [84]:

def objective(trial):
  num_hidden_layers=trial.suggest_int('num_hidden_layers',1,5)
  neurons_per_layer=trial.suggest_int('neurons_per_layer',8,128,step=8)
  epochs=trial.suggest_int('epochs',10,50,step=10)
  learning_rate=trial.suggest_float("learning_rate",1e-5,1e-1,log=True)
  dropout_rate=trial.suggest_float("dopout_rate",0.1,0.5,step=0.1)
  batch_size=trial.suggest_categorical("batch_size",(16,32,64,128))
  optimizer_name=trial.suggest_categorical("optimizer",['Adam','SGD','RMSprop'])
  weight_decay=trial.suggest_float('weight_decay',1e-5,1e-3,log=True)
  train=DataLoader(train_dataset,batch_size=batch_size,shuffle=True,pin_memory=True)
  test=DataLoader(test_dataset,batch_size=batch_size,shuffle=False,pin_memory=True)
  input_dim=784
  output_dim=10
  model=myNN(input_dim,output_dim,num_hidden_layers,neurons_per_layer,dropout_rate)
  model.to(device)


  criterion=nn.CrossEntropyLoss()

  if optimizer_name=='Adam':
        optimiser=optim.Adam(model.parameters(),lr=learning_rate,weight_decay=weight_decay)
  elif optimizer_name=='SGD':
      optimiser=optim.SGD(model.parameters(),lr=learning_rate,weight_decay=weight_decay)
  else:
        optimiser=optim.RMSprop(model.parameters(),lr=learning_rate,weight_decay=weight_decay)
  for epoch in range(epochs):

    for batch_features,batch_labels in train:
      batch_features,batch_labels=batch_features.to(device),batch_labels.to(device)
      output= model(batch_features)
      loss=criterion(output,batch_labels)
      optimiser.zero_grad()
      loss.backward()
      optimiser.step()

  model.eval()
  total=0
  correct=0
  with torch.no_grad():
    for batch_features,batch_labels in test:
      batch_features,batch_labels=batch_features.to(device),batch_labels.to(device)
      output= model(batch_features)
      _, predicted=torch.max(output,1)
      total=total+batch_labels.shape[0]
      correct+=(predicted==batch_labels).sum().item()
  print(correct)
  print("Accuracy:", correct / total)



  return correct/total

In [85]:
import optuna
study=optuna.create_study(direction='maximize')


[I 2026-01-09 17:34:39,442] A new study created in memory with name: no-name-3c7fc791-33c7-4b50-b6dc-486c07ce8bb3


In [86]:
study.optimize(objective,n_trials=50)

[I 2026-01-09 17:34:42,166] Trial 0 finished with value: 0.7883333333333333 and parameters: {'num_hidden_layers': 1, 'neurons_per_layer': 24, 'epochs': 20, 'learning_rate': 0.013488741812504453, 'dopout_rate': 0.4, 'batch_size': 128, 'optimizer': 'Adam', 'weight_decay': 1.0898150019468224e-05}. Best is trial 0 with value: 0.7883333333333333.


946
Accuracy: 0.7883333333333333


[I 2026-01-09 17:34:46,977] Trial 1 finished with value: 0.61 and parameters: {'num_hidden_layers': 1, 'neurons_per_layer': 96, 'epochs': 30, 'learning_rate': 8.208365793205529e-05, 'dopout_rate': 0.2, 'batch_size': 64, 'optimizer': 'SGD', 'weight_decay': 4.0064015356399415e-05}. Best is trial 0 with value: 0.7883333333333333.


732
Accuracy: 0.61


[I 2026-01-09 17:34:51,227] Trial 2 finished with value: 0.7583333333333333 and parameters: {'num_hidden_layers': 3, 'neurons_per_layer': 128, 'epochs': 10, 'learning_rate': 0.027725463485145336, 'dopout_rate': 0.1, 'batch_size': 32, 'optimizer': 'Adam', 'weight_decay': 7.508708498232789e-05}. Best is trial 0 with value: 0.7883333333333333.


...

969
Accuracy: 0.8075


[I 2026-01-09 17:42:04,189] Trial 48 finished with value: 0.7475 and parameters: {'num_hidden_layers': 3, 'neurons_per_layer': 96, 'epochs': 40, 'learning_rate': 0.03727459747158958, 'dopout_rate': 0.30000000000000004, 'batch_size': 64, 'optimizer': 'RMSprop', 'weight_decay': 7.26921666246633e-05}. Best is trial 45 with value: 0.84.


897
Accuracy: 0.7475


[I 2026-01-09 17:42:09,092] Trial 49 finished with value: 0.81 and parameters: {'num_hidden_layers': 1, 'neurons_per_layer': 96, 'epochs': 30, 'learning_rate': 0.004377709257256069, 'dopout_rate': 0.30000000000000004, 'batch_size': 64, 'optimizer': 'RMSprop', 'weight_decay': 3.117563936404302e-05}. Best is trial 45 with value: 0.84.


972
Accuracy: 0.81


In [88]:
study.best_value

0.84

In [89]:
study.best_params

{'num_hidden_layers': 3,
 'neurons_per_layer': 96,
 'epochs': 50,
 'learning_rate': 0.0009665177603765672,
 'dopout_rate': 0.30000000000000004,
 'batch_size': 64,
 'optimizer': 'RMSprop',
 'weight_decay': 5.0136702111136634e-05}